# Who's the Best NCAA Tournament Coach?

With the NCAA Men's basketball tournament and my bracket officially busted I want to start gathering information for next year. I unfortunately chose Virginia to win it all over Villanova as I thought they were the best teams all year long. I was right about one team but woefully wrong about the other. Congrats to UMBC but I want to win my bracket challenge next year.

One aspect I have thought to include in my choices next year is the coach of the team. It seems to me that the same coaches are in the sweet sixteen and beyond each year (Bill Self, Coach K, Jim Boeheim, etc). Also it seems certain coaches get their teams to overacheive (Beilein, Brad Stevens, etc.). My goal is to find current coaches that win more than they are supposed to in the tournament.

To do this I have taken the data from the [Kaggle NCAA 2018 ML Contest](https://www.kaggle.com/c/mens-machine-learning-competition-2018) and will analyze it to find the average amount of wins for each seed and compare that to how the coaches do.

## Import, Combine, and Clean the Data

To start I import the data for the conferences, teams, and coaches. Then I merge (AKA SQL join) and sort them by team to get data for each team, their coach, and the current conference on a yearly basis. 

In [53]:
%matplotlib inline
import matplotlib as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scipy as spy
from sklearn.model_selection import train_test_split

conf = pd.read_csv("TeamConferences.csv")
teams = pd.read_csv("Teams.csv")
coaches = pd.read_csv("TeamCoaches.csv")

allTeams = coaches.merge(teams[["TeamID", "TeamName"]], on="TeamID").merge(conf, on=["Season", "TeamID"])
allTeams.drop(["FirstDayNum","LastDayNum"], axis=1, inplace=True)
allTeams.drop_duplicates(inplace=True)
allTeams.head(25)


,Season,TeamID,CoachName,TeamName,ConfAbbrev
0,1985,1102,reggie_minton,Air Force,wac
1,1986,1102,reggie_minton,Air Force,wac
2,1987,1102,reggie_minton,Air Force,wac
3,1988,1102,reggie_minton,Air Force,wac
4,1989,1102,reggie_minton,Air Force,wac
5,1990,1102,reggie_minton,Air Force,wac
6,1991,1102,reggie_minton,Air Force,wac
7,1992,1102,reggie_minton,Air Force,wac
8,1993,1102,reggie_minton,Air Force,wac
9,1994,1102,reggie_minton,Air Force,wac


Next I will import the compact results as well as the tournament seeds files. Then merge the results with the seeds to create a data frame with all of the results as well as the winning seed and losing seed. I also had to clean the seed values as the seeds included a letter denoting which region the seed was in. All I cared about was the number of the seed, for more explanation and description of the function used see the [Basic Starter Kernel - NCAA Men's Dataset](https://www.kaggle.com/juliaelliott/basic-starter-kernel-ncaa-men-s-dataset).

In [54]:
results = pd.read_csv("NCAATourneyCompactResults.csv")
seeds = pd.read_csv("NCAATourneySeeds.csv")
import re

# convert and remove string characters from seed values
# borrowed from the Kaggle NCAA 2018 Men's competition tutorial notebook.
def seed_to_int(seed):
    #Get just the digits from the seeding. Return as int
    s_int = int(re.sub(r'[A-z]', '', seed))
    return s_int


seeds['seed_int'] = seeds.Seed.apply(seed_to_int)
seeds.drop(labels=['Seed'], inplace=True, axis=1)

seedResults =  results.merge(seeds, right_on=["TeamID","Season"], left_on= ["WTeamID","Season"]).merge(seeds, right_on=["TeamID","Season"], left_on= ["LTeamID","Season"])
seedResults.drop(["TeamID_x", "TeamID_y"],axis = 1, inplace=True)
seedResults.rename(index=str, columns={"seed_int_x":"WSeed", "seed_int_y":"LSeed"}, inplace=True)
seedResults.tail(10)



,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WSeed,LSeed
2170,2018,137,1345,76,1139,73,N,0,2,10
2171,2018,135,1393,57,1395,52,N,0,11,6
2172,2018,137,1393,55,1277,53,N,0,11,3
2173,2018,135,1401,73,1344,69,N,0,7,10
2174,2018,137,1401,86,1314,65,N,0,7,2
2175,2018,135,1420,67,1438,54,N,0,16,1
2176,2018,135,1452,85,1293,68,N,0,5,12
2177,2018,137,1452,94,1267,71,N,0,5,13
2178,2018,135,1267,81,1455,75,N,0,13,4
2179,2018,135,1462,102,1411,83,N,0,1,16


Next I needed to merge the seed results with the team information

In [55]:
allResults =  seedResults.merge(allTeams, right_on=["TeamID","Season"], left_on= ["WTeamID","Season"]).merge(allTeams, right_on=["TeamID","Season"], left_on= ["LTeamID","Season"])

def xyToWL(colNames):
    """
    Convert suffix _x and _y to prefix W and L respectively.

    Parameters
    ----------
    colNames : list
        List of column names to convert

    Returns
    -------
    list
        New column names converted to WColumnName from ColumnName_x

    """
    newCols = []
    for col in colNames:
        if col.endswith("_x"):
            newCol = col.replace("_x", "")
            newCol = "W" + newCol
            newCols.append(newCol)
        elif col.endswith("_y"):
            newCol = col.replace("_y", "")
            newCol = "L" + newCol
            newCols.append(newCol)
        else:
            newCols.append(col)
    return newCols

# drop repeated w team id and l team id
allResults.drop(["TeamID_x", "TeamID_y"], axis=1, inplace=True)
allResults.drop_duplicates(inplace=True)

# Convert suffixed columns
allResults.columns = xyToWL(allResults.columns)
allResults = allResults[~((allResults.WSeed ==16) & (allResults.LSeed == 16))]
allResults.sort_values(["Season","DayNum"], inplace=True)
allResults.tail(15)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WSeed,LSeed,WCoachName,WTeamName,WConfAbbrev,LCoachName,LTeamName,LConfAbbrev
2156,2018,141,1260,69,1305,68,N,0,11,7,porter_moser,Loyola-Chicago,mvc,eric_musselman,Nevada,mwc
2160,2018,141,1276,99,1401,72,N,0,3,7,john_beilein,Michigan,big_ten,billy_kennedy,Texas A&M,sec
2184,2018,141,1243,61,1246,58,N,0,9,5,bruce_weber,Kansas St,big_twelve,john_calipari,Kentucky,sec
2188,2018,141,1199,75,1211,60,N,0,9,4,leonard_hamilton,Florida St,acc,mark_few,Gonzaga,wcc
2143,2018,142,1181,69,1393,65,N,0,2,11,mike_krzyzewski,Duke,acc,jim_boeheim,Syracuse,acc
2150,2018,142,1242,80,1155,76,N,0,1,5,bill_self,Kansas,big_twelve,brad_brownell,Clemson,acc
2169,2018,142,1403,78,1345,65,N,0,3,2,chris_beard,Texas Tech,big_twelve,matt_painter,Purdue,big_ten
2172,2018,142,1437,90,1452,78,N,0,1,5,jay_wright,Villanova,big_east,bob_huggins,West Virginia,big_twelve
2157,2018,143,1260,78,1243,62,N,0,11,9,porter_moser,Loyola-Chicago,mvc,bruce_weber,Kansas St,big_twelve
2161,2018,143,1276,58,1199,54,N,0,3,9,john_beilein,Michigan,big_ten,leonard_hamilton,Florida St,acc


Next I needed to calculate the expected wins (EWins) for each seed. To do this I grouped and merged the `allResults` dataframe on the seeds to give the `totalWins`, `totalLosses`, and `WPct` for each seed. I then calculated the `EWins` using the following formula:

$$EWins = \frac{totalWins}{4 \cdot numberOfSeasons}$$ 

*Note: The 4 in the denominator is due to the fact that each seed has 4 teams per year.*

In [56]:
# helper function mergeWL
def mergeWL(resDF, onCol):
    '''
    Group resDF by onCol and count totalWins, total Losses and WPct
    
    Parameters
    ----------
    resDF : pandas.dataframe
        Dataframe of results to filter and group wins and losses
        
    onCol: str
        String of column to group by

    Returns
    -------
    pandas.dataframe
        New dataframe with wins, losses, and WPct calculated.

    '''
    wins = resDF.groupby("W"+onCol, as_index=False)["WTeamID"].agg(['count']).rename(columns={'count': 'totalWins'})
    loss = resDF.groupby("L"+onCol, as_index=False)["LTeamID"].agg(['count']).rename(columns={'count': 'totalLosses'})
    comb = wins.merge(loss, how="right", left_index = True, right_index = True)
    comb.reset_index(inplace=True)
    comb.rename(columns={comb.columns[0]:onCol}, inplace=True)
    comb["WPct"] = comb.totalWins/(comb.totalWins + comb.totalLosses)
    comb.fillna(0, inplace=True)
    return comb

In [57]:
bySeed = mergeWL(allResults, "Seed")
# EWins is yearly expected wins per team by their seed. --> totalWins / 4 * number of seasons   (4 due to 4 different seeds every year)
bySeed["EWins"] = bySeed.totalWins/(4*seedResults.Season.nunique())
bySeed = bySeed.round(decimals=3)
bySeed.head(16)


,Seed,totalWins,totalLosses,WPct,EWins
0,1,457,118,0.795,3.360
1,2,327,134,0.709,2.404
2,3,255,135,0.654,1.875
3,4,210,135,0.609,1.544
4,5,152,137,0.526,1.118
5,6,151,137,0.524,1.110
6,7,128,136,0.485,0.941
7,8,98,136,0.419,0.721
8,9,81,136,0.373,0.596
9,10,84,138,0.378,0.618


Finally on to calculating coach wins, losses, and the average of their wins added based on their seed (`AvgWinsAdded`). To start I grouped and merged `allResults` by `CoachName`.

In [58]:
byCoach = mergeWL(allResults, "CoachName")
byCoach.head(10)

,CoachName,totalWins,totalLosses,WPct
0,al_brown,0.0,1,0.000000
1,al_skinner,8.0,9,0.470588
2,alan_leforce,1.0,2,0.333333
3,andrew_toole,0.0,1,0.000000
4,andy_enfield,4.0,3,0.571429
5,andy_kennedy,2.0,2,0.500000
6,andy_russo,2.0,2,0.500000
7,andy_stoglin,0.0,2,0.000000
8,anthony_evans,1.0,1,0.500000
9,anthony_grant,1.0,3,0.250000


Next I calculated the average seed (`avgSeed`) and number of appearances (`Appearances`) for each coach.

Finally I needed to calculate the number of wins added per year by each coach. To do this I created another dataframe called `byCoachSeed` to hold all of the wins and losses grouped by coach and season. Using this dataframe I was able to count the number of wins each year by each team (coach) to get their wins in the tournament for that year. Also I calculated the difference between average wins by seed and the actual wins that year for that coach(`Diff`).

In [59]:
winCoachSeed = allResults.groupby(["WCoachName", "Season"], as_index=False)["WSeed"].mean()
lossCoachSeed = allResults.groupby(["LCoachName", "Season"], as_index=False)["LSeed"].mean()
winCoachSeed.rename(columns={"WCoachName":"CoachName","WSeed":"Seed"}, inplace=True)
lossCoachSeed.rename(columns={"LCoachName":"CoachName","LSeed":"Seed"}, inplace=True)
byCoachSeed = winCoachSeed.append(lossCoachSeed,ignore_index=True).drop_duplicates().sort_values(["CoachName", "Season"])
byCoachSeed.reset_index(drop=True, inplace=True)

coachWins = allResults.groupby(["WCoachName", "Season"], as_index=False)["WSeed"].count().rename(columns={"WSeed":"TourneyWins","WCoachName":"CoachName"})
byCoachSeed = byCoachSeed.merge(coachWins, how="left",on=["CoachName", "Season"]).fillna(0)
byCoachSeed["TourneyWins"] = byCoachSeed["TourneyWins"].astype(int)

byCoachSeed = byCoachSeed.merge(bySeed[["Seed", "EWins"]], how="left", on="Seed").fillna(0)
byCoachSeed["Diff"] = byCoachSeed.TourneyWins - byCoachSeed.EWins
byCoachSeed.head(15)



,CoachName,Season,Seed,TourneyWins,EWins,Diff
0,al_brown,1986,14,0,0.176,-0.176
1,al_skinner,1993,8,1,0.721,0.279
2,al_skinner,1997,9,0,0.596,-0.596
3,al_skinner,2001,3,1,1.875,-0.875
4,al_skinner,2002,11,0,0.699,-0.699
5,al_skinner,2004,6,2,1.110,0.890
6,al_skinner,2005,4,1,1.544,-0.544
7,al_skinner,2006,4,2,1.544,0.456
8,al_skinner,2007,7,1,0.941,0.059
9,al_skinner,2009,7,0,0.941,-0.941


In [60]:
avgSeed = byCoachSeed.groupby("CoachName", as_index=False).agg({'Seed': "mean", 'Season': 'count'})
avgSeed.rename(columns={"Seed":"AvgSeed", "Season":"Appearances"}, inplace=True)
byCoach = byCoach.merge(avgSeed,on="CoachName").round(3)
byCoach["totalWins"] = byCoach["totalWins"].astype(int)
byCoach.head(10)

,CoachName,totalWins,totalLosses,WPct,AvgSeed,Appearances
0,al_brown,0,1,0.000,14.000,1
1,al_skinner,8,9,0.471,6.556,9
2,alan_leforce,1,2,0.333,12.000,2
3,andrew_toole,0,1,0.000,16.000,1
4,andy_enfield,4,3,0.571,11.333,3
5,andy_kennedy,2,2,0.500,11.500,2
6,andy_russo,2,2,0.500,8.500,2
7,andy_stoglin,0,2,0.000,16.000,2
8,anthony_evans,1,1,0.500,15.000,1
9,anthony_grant,1,3,0.250,10.333,3


To get the `AvgWinsAdded` for each coach I was able to group `byCoachSeed` from above on the coaches 

In [61]:
coachTotalEWins = byCoachSeed.groupby("CoachName", as_index=False)["Diff"].mean()
byCoach = byCoach.merge(coachTotalEWins, how="left", on="CoachName").rename(columns={"Diff":"AvgWinsAdded"})
byCoach.sort_values("AvgWinsAdded",ascending=False, inplace=True)
byCoach.reset_index(drop=True, inplace=True)
byCoach.head(15)


,CoachName,totalWins,totalLosses,WPct,AvgSeed,Appearances,AvgWinsAdded
0,porter_moser,4,1,0.800,11.0,1,3.3010
1,john_giannini,3,1,0.750,13.0,1,2.7430
2,kevin_ollie,7,1,0.875,8.0,2,2.7315
3,jim_rosborough,5,1,0.833,2.0,1,2.5960
4,kevin_mackey,2,1,0.667,14.0,1,1.8240
5,brad_stevens,12,5,0.706,7.0,5,1.5028
6,rollie_massimino,11,4,0.733,9.0,5,1.4866
7,jim_brandenburg,2,1,0.667,12.0,1,1.4780
8,darrin_horn,2,1,0.667,12.0,1,1.4780
9,joe_b_hall,2,1,0.667,12.0,1,1.4780


Kevin Ollie leads the pack! He took a 7 seed to the Final Four and won it the next year. However I am more focused on coaches that have more lasting impact. Let's limit it to 3 tournament appearances.


### Top 50 Coaches by Average Wins Above  Seed 

In [62]:
byCoach3App = byCoach[byCoach["Appearances"] >= 3].reset_index(drop=True)
byCoach3App.head(50)

,CoachName,totalWins,totalLosses,WPct,AvgSeed,Appearances,AvgWinsAdded
0,brad_stevens,12,5,0.706,7.000,5,1.502800
1,rollie_massimino,11,4,0.733,9.000,5,1.486600
2,john_beilein,24,12,0.667,7.417,12,0.930833
3,larry_brown,13,4,0.765,4.200,5,0.885400
4,richard_williams,6,3,0.667,5.000,3,0.882000
5,john_groce,4,3,0.571,11.333,3,0.875333
6,andy_enfield,4,3,0.571,11.333,3,0.838000
7,frank_martin,10,5,0.667,6.600,5,0.823400
8,billy_donovan,35,12,0.745,4.071,14,0.734286
9,sonny_smith,7,5,0.583,9.400,5,0.723200


### Bottom 50 Coaches 

In [63]:
byCoach3App.tail(50)

,CoachName,totalWins,totalLosses,WPct,AvgSeed,Appearances,AvgWinsAdded
202,mick_cronin,6,10,0.375,7.800,10,-0.348500
203,dave_odom,10,9,0.526,5.222,9,-0.349778
204,seth_greenberg,1,3,0.250,9.667,3,-0.358000
205,bobby_lutz,2,5,0.286,7.800,5,-0.369400
206,mike_montgomery,18,16,0.529,6.188,16,-0.378188
207,mark_fox,2,5,0.286,8.200,5,-0.378200
208,kevin_stallings,6,9,0.400,6.778,9,-0.393111
209,wayne_tinkle,0,4,0.000,11.750,4,-0.407750
210,tom_asbury,0,4,0.000,12.250,4,-0.417250
211,fran_dunphy,3,16,0.158,10.625,16,-0.417875


## Conclusion 

Looking at the Top 50 and bottom 50 provide some insights. Tony Bennett is in the bottom 10% (234 out of 247). Maybe defense doesn't win championships? Also if Pitt wasn't happy with Jamie Dixon and his teams' tournament performance, Kevin Stallings might not have been much of an improvement. On the positive side I can see why Brad Stevens has had success in both NCAA and NBA as he leads the group of coaches with 3 or more appearances. Also a lot of the legends (Izzo, Pitino, Calipari) are up towards the top as well.

Some teams to watch next year if they make the tournament could be Indiana (Archie Miller), South Carolina (Frank Martin), and Michigan (John Beilein). It would be nice to add the results from this year's tournament as some of these values might change. Beilein would improve as he took #3 Michigan to the Final and Tony Bennett will fall even further.

**Update** I added the data for the 2018 tournament and John Beilein did indeed improve all the way up to the number three spot. Unfortunately Tony Bennett dropped significantly due to losing in the first round as a number one seed.